In [107]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas para manipulación y análisis de datos en Python.
# Importar librerías para web scraping y manipulación de datos

from bs4 import BeautifulSoup
import requests
# Importar librerías para automatización de navegadores web con Selenium

from selenium import webdriver  
from webdriver_manager.chrome import ChromeDriverManager  
# ChromeDriverManager gestiona la instalación del controlador de Chrome

from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
# Importar librerías para pausar la ejecución
# -----------------------------------------------------------------------
from time import sleep  # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.

import random
from datetime import datetime
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings("ignore")


In [4]:
df_1 = pd.read_csv("INFO_PELIS/cortos_2000_2006.csv", index_col = 0)
df_2 = pd.read_csv("INFO_PELIS/cortos_2021.csv", index_col = 0)
df_3 = pd.read_csv("INFO_PELIS/cortos_2022.csv", index_col = 0)
df_4 = pd.read_csv("INFO_PELIS/cortos_2023.csv", index_col = 0)
df_5 = pd.read_csv("INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)
df_6 = pd.read_csv("INFO_PELIS/pelis_2000_2006.csv", index_col = 0)
df_7 = pd.read_csv("INFO_PELIS/pelis_2007_2010.csv", index_col = 0)
df_8 = pd.read_csv("INFO_PELIS/pelis_2011_2013.csv", index_col = 0)
df_9 = pd.read_csv("INFO_PELIS/cortos_2007_2008.csv", index_col = 0)
df_10 = pd.read_csv("INFO_PELIS/cortos_2009_2010.csv", index_col = 0)
df_11 = pd.read_csv("INFO_PELIS/cortos_2011.csv", index_col = 0)
df_12 = pd.read_csv("INFO_PELIS/cortos_2012.csv", index_col = 0)
df_13 = pd.read_csv("INFO_PELIS/cortos_2013.csv", index_col = 0)
df_14 = pd.read_csv("INFO_PELIS/lista_pelis_2014_2019.csv", index_col = 0)
df_15 = pd.read_csv("INFO_PELIS/cortos_2014_2019.csv", index_col = 0)



In [6]:
df_15.head()

,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,Short,Le sacre d'automne,2014,11.0,tt0165933
1,Short,Eloise,2014,NaN,tt0839791
2,Short,Cocoon,2014,8.0,tt10005668
3,Short,"Zahtevam most, pripada mi!",2014,9.0,tt10010244
4,Short,Nothing is as it seems,2014,NaN,tt10010624


In [5]:
df_peliculas = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15], axis=0)

In [8]:
df_peliculas.head()

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874
1,Drama,Short,Nights,2000,3.0,tt0138655
2,Drama,Short,Caught,2000,8.0,tt0142155
3,Drama,Short,Smog,2000,3.0,tt0144613
4,Drama,Short,Safe Journey,2000,NaN,tt0188189


In [8]:
df_peliculas.iloc[[25024]]

,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
2,Drama,Short,Welcome to Mumbai,2021,3.0,tt10040326


In [9]:
df_peliculas.tail()


,Genero,Tipo,Titulo,Año de estreno,Mes de estreno,ID
175153,NaN,Short,Temptation (Eghva),2019,9.0,tt30625971
175154,NaN,Short,Hard Way Out,2019,8.0,tt30701899
175155,NaN,Short,The Battle of the Sunset,2019,2.0,tt30766508
175156,NaN,Short,Bear Squad,2019,6.0,tt30796814
175157,NaN,Short,Dream Fast,2019,5.0,tt31186006


In [12]:
df_pelis_ordenado = df_2.sort_values(by="Año de estreno")

20331

In [7]:
df_pelis_ordenado.to_csv("pelis_ordenadas.csv")

In [8]:
lista_titulos= df_pelis_ordenado["Titulo"].to_list()

In [10]:
lista_años= df_pelis_ordenado["Año de estreno"].to_list()

In [11]:
list_busqueda_rotten = []
for titulo,año in zip(lista_titulos,lista_años):
    list_busqueda_rotten.append({titulo:año})


In [12]:
lista_tipo= df_pelis_ordenado["Tipo"].to_list()
cortos= 0
pelis = 0

for elemento in lista_tipo:
    if elemento == "Short":
        cortos += 1
    elif elemento == "Movie":
        pelis +=1

print(cortos,pelis)

#30% pelis y 70%cortos

380488 171253


In [6]:
lista_ID = df_pelis_ordenado["ID"].to_list()

In [14]:
lista1=[df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15]

suma = 0
for e in lista1:
    suma += len(e)
print("Longitus suma de los data",suma)   
print("Longitud DATAFRAME", len(lista_ID)) 


Longitus suma de los data 551741
Longitud DATAFRAME 551741


In [32]:
union_peliculas = pd.read_csv("union_peliculas.csv")
lista_ID = union_peliculas["ID"].to_list()
id = lista_id

In [37]:
type(lista_ID)

list

In [115]:
def get_url(driver, path, accept_cookies = False): 

    url = f"https://www.imdb.com/{path}"

    print(f"getting url with {path}...")
    driver.get(url)
    print("url loaded")
    # driver.maximize_window()
    if accept_cookies:
        sleep(2)
        # Aceptar las cookies PAG IMDB
        driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
        print("cookies accepted")

    return driver
    


In [98]:
def getting_name_score(driver, id):
    
    #nombre:
    try:
        name = driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text
    except:
        name = f"Error nombre para la pelicula con id: {id}"
                
    #puntuacion:
        
    try: 
        score = driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text
                
    except:  
        score = "--"

    return name, score

In [113]:
def getting_directors_scripters(driver):
        directors = "DESCONOCIDO"
        scripters = "DESCONOCIDO"

        for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
        
            try:
                nombre = elemento.find_element("css selector", "span").text
                    
                if nombre == "Dirección":
                

                    # Seleccionar director
                    try:          
                        directors = [e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""]
                    except:
                        directors = "DESCONOCIDO"
                    

                elif nombre == "Guión":
                    
                    
                    # Seleccionar Guionista
                    try:          
                        scripters = [e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""]
                    except:
                        scripters = "DESCONOCIDO"
            except:
                continue


        return directors, scripters

In [100]:
def getting_plot(driver):

    try:      
        return driver.find_element("css selector", 'span[data-testid="plot-xl"]').text
                    
    except:
            
        return "DESCONOCIDO"
    

In [101]:
def getting_time(driver):
            
    # Seleccionar DURACION        
    try:   
        
        busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
            
        if "min" in busqueda_duracion or "h" in busqueda_duracion:
            return busqueda_duracion
                
        else:
            return "--"
        
    except:
        
        return "--"

In [102]:
def getting_cast(driver, dict_actores):
    try:
        for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
            actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
            
            for element in actor[0:10]:
            
                nombre = element.text
                    
                if nombre not in dict_actores.keys():
                    
                    dict_actores[nombre]  = element.get_attribute("href")

                    driver.find_element("css selector",'section[data-testid="title-cast"]')
            
    except:
        dict_actores[nombre]  = "desconocido"
    return dict_actores

In [129]:
execution_number = 1
counter = 25122
incremental = 100

options = webdriver.ChromeOptions()
options.page_load_strategy = "eager"
driver = webdriver.Chrome(options)

driver = get_url(driver, "", True)

films_dict = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}
actors_dict = {}
# diccionario_actores = {"Actor":[], "Fehca_nacimiento":[], "Conocido_por:":[], "Que_hace":[], "Premios":[]}
df_films = pd.DataFrame(films_dict)

errors = 0

times_dict = { "id": [], "time_sec": []}

for id in lista_ID[counter:counter+incremental]:
    start_time = datetime.now()
    try:
        driver = get_url(driver, f"title/{id}")
    except:
        print(f"Error getting {id}")
        if errors > 10:
            print(f"Fatal error")
            break
        errors += 1
        sleep(random.randint(2,20))

    errors = 0
    sleep(2)
    name, score = getting_name_score(driver, id)
    films_dict["Nombre de la película"].append(name)
    films_dict["Puntuacion_IMDB"].append(score)
    directors, scripters = getting_directors_scripters(driver)
    films_dict["Dirección"].append(directors)
    films_dict["Guionistas"].append(scripters)
    films_dict["Argumento"].append(getting_plot(driver))
    films_dict["Duración"].append(getting_time(driver))
    actors_dict = getting_cast(driver, actors_dict)

    if counter % 10 == 0:
        df_films = pd.DataFrame(films_dict)
        df_films.to_csv(f"patri_films_{counter}.csv")
    
    counter += 1
    end_time = datetime.now()
    times_dict["id"].append(id)
    times_dict["time_sec"].append((end_time-start_time).seconds)
    print(f"time to obtain {id} was {(end_time-start_time).seconds} seconds")
    # break

df_times = pd.DataFrame(times_dict)
df_times.to_csv(f"{execution_number}_times_extraction.csv")
df_films = pd.DataFrame(films_dict)
df_films.to_csv(f"{execution_number}_patri_films_total.csv")
df_actors = pd.DataFrame({"name": actors_dict.keys(), "link": actors_dict.items()})
df_actors.to_csv(f"{execution_number}_patri_actors_total.csv")

getting url with ...
url loaded
cookies accepted
getting url with title/tt10709592...
url loaded
time to obtain tt10709592 was 59 seconds
getting url with title/tt10716904...
Error getting tt10716904


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.94)
Stacktrace:
0   chromedriver                        0x00000001033b453c chromedriver + 3966268
1   chromedriver                        0x00000001033acac8 chromedriver + 3934920
2   chromedriver                        0x000000010302fda0 chromedriver + 277920
3   chromedriver                        0x000000010300afb0 chromedriver + 126896
4   chromedriver                        0x0000000103097414 chromedriver + 701460
5   chromedriver                        0x00000001030aa304 chromedriver + 779012
6   chromedriver                        0x0000000103066fb0 chromedriver + 503728
7   chromedriver                        0x0000000103067a28 chromedriver + 506408
8   chromedriver                        0x0000000103379724 chromedriver + 3725092
9   chromedriver                        0x000000010337dc18 chromedriver + 3742744
10  chromedriver   

In [126]:
df_times[["time_sec"]].mean()

time_sec    108.94
dtype: float64

In [75]:
print(diccionario_peliculas)

{'Puntuacion_IMDB': ['--', '--'], 'Dirección': [], 'Guionistas': [], 'Argumento': [], 'Duración': [], 'Nombre de la película': ['Error nombre para la pelicula con id: tt10011052', 'Error nombre para la pelicula con id: tt10011052']}


In [25]:
contador = 25022
maximo = 25122
intentos = 0 
antiguo_contador = 0 


def getting_name_puntuaction(id):
    
    #nombre:
    try:
        diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text)
    except:
        diccionario_peliculas["Nombre de la película"].append(f"Error nombre para la pelicula con id: {id}")
                
    #puntuacion:
        
    try: 
        diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text)
                
    except:  
        diccionario_peliculas["Puntuacion_IMDB"].append("--")

def getting_director_script(id):
    #direccion y guion: 
        existe_direccion = False
        existe_guion = False

        for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
        
            try:
                nombre = elemento.find_element("css selector", "span").text
                    
                if nombre == "Dirección":
                
                    existe_direccion = True

                    # Seleccionar director
                    try:          
                        diccionario_peliculas["Dirección"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                    except:
                        diccionario_peliculas["Dirección"].append("DESCONOCIDO") 
                    

                elif nombre == "Guión":
                    
                    existe_guion = True
                    
                    # Seleccionar Guionista
                    try:          
                        diccionario_peliculas["Guionistas"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                    except:
                        diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
            except:
                continue

            if existe_direccion == False:    
                diccionario_peliculas["Dirección"].append("DESCONOCIDO")
            if existe_guion == False:
                diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
            

            # sleep(2)
            # Seleccionar ARGUMENTO
    
    def getting_plot(id):

        try:      
            diccionario_peliculas["Argumento"].append(driver.find_element("css selector", 'span[data-testid="plot-xl"]').text)
                    
        except:
            
            diccionario_peliculas["Argumento"].append("Desconocido")
    def getting_time(id):
            
        # Seleccionar DURACION        
        try:   
            busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
            
            if "min" in busqueda_duracion or "h" in busqueda_duracion:
                diccionario_peliculas["Duración"].append(busqueda_duracion)
                
            else:
                diccionario_peliculas["Duración"].append("--")
        except:
            diccionario_peliculas["Duración"].append("--")

            # Buscar los 10 primeros actores    
            try:
                for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
                    actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
            
                    for element in actor[0:10]:
                        nombre = element.text
                    
                        if nombre not in dict_actores.keys():
                            dict_actores[nombre]  = element.get_attribute("href")

                            driver.find_element("css selector",'section[data-testid="title-cast"]')
            
            except:
                    dict_actores[nombre]  = "desconocido"
        
            contador += 1

    except:
        print(f"ha fallado la pelicula {contador}")
        if antiguo_contador == contador:
            intentos += 1
            if intentos == 4:
                sleep(random.randint(2,20))
                intentos = 0
                driver.back()
                contador += 1
        else:
            intentos += 1
            antiguo_contador = contador



   

### REVISAR:
# Lista de tuplas

getting url...
url loaded
cookies accepted
searching film $tt10011052
searching film $tt10033714
searching film $tt10040326
searching film $tt10048746
searching film $tt10049134
searching film $tt10049710
searching film $tt10074108
searching film $tt10116030
searching film $tt10116000
searching film $tt10126104
searching film $tt10121942
searching film $tt10130050
searching film $tt10136966
searching film $tt10137298
searching film $tt10160440
searching film $tt10160712
searching film $tt10160948
searching film $tt10162788
searching film $tt10165720
searching film $tt10173624
ha fallado la pelicula 25041
searching film $tt10173624
ha fallado la pelicula 25041
searching film $tt10173624
ha fallado la pelicula 25041
searching film $tt10173624
ha fallado la pelicula 25041
searching film $tt10178240
searching film $tt10202368
searching film $tt10209778
searching film $tt10211236
searching film $tt10214926
searching film $tt10217312
searching film $tt10223204
searching film $tt10232622
sear

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.94)
Stacktrace:
0   chromedriver                        0x00000001008cc53c chromedriver + 3966268
1   chromedriver                        0x00000001008c4ac8 chromedriver + 3934920
2   chromedriver                        0x0000000100547da0 chromedriver + 277920
3   chromedriver                        0x0000000100522fb0 chromedriver + 126896
4   chromedriver                        0x00000001005af414 chromedriver + 701460
5   chromedriver                        0x00000001005c2304 chromedriver + 779012
6   chromedriver                        0x000000010057efb0 chromedriver + 503728
7   chromedriver                        0x000000010057fa28 chromedriver + 506408
8   chromedriver                        0x0000000100891724 chromedriver + 3725092
9   chromedriver                        0x0000000100895c18 chromedriver + 3742744
10  chromedriver   

In [16]:
diccionario_peliculas.keys()
for k,v in diccionario_peliculas.items():
    
    print(k,len(diccionario_peliculas[k]))

Puntuacion_IMDB 0
Dirección 0
Guionistas 0
Argumento 0
Duración 0
Nombre de la película 0


In [17]:
len(dict_actores)

0

In [26]:
## Convertir en DF Pelis y Actores
df_peliculas_guardar = pd.DataFrame(diccionario_peliculas)


df_actores_guardar = pd.Series(dict_actores,name="url")
df_actores_guardar.index.name="nombre_actor"
df_actores_guardar.reset_index()

,nombre_actor,url
0,Camila Duarte,https://www.imdb.com/name/nm10543478/?ref_=tt_...
1,Virgo Martinez,https://www.imdb.com/name/nm10543479/?ref_=tt_...
2,Luciano Antonino,https://www.imdb.com/name/nm10027758/?ref_=tt_...
3,Bianca Carolina,https://www.imdb.com/name/nm10543482/?ref_=tt_...
4,Domenica Centeno,https://www.imdb.com/name/nm10543480/?ref_=tt_...
...,...,...
500,Devin Lee White,https://www.imdb.com/name/nm6751991/?ref_=tt_c...
501,Patric James Callahan,https://www.imdb.com/name/nm9501053/?ref_=tt_c...
502,James Ciskanik,https://www.imdb.com/name/nm7192999/?ref_=tt_c...
503,Cheray O'Neal,https://www.imdb.com/name/nm0641886/?ref_=tt_c...


In [27]:
## GUARDAR EN CSV la info
df_peliculas_guardar.to_csv("pelis_patri_25022_25122.csv")
df_actores_guardar.to_csv("actores_patri_25022_25122.csv")


In [22]:
#Busqueda actores                 ##incluido en el codigo##

driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

# dict_actores = {"actor":[],"url":[]}
dict_actores={}


lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
        actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
 
        for element in actor[0:10]:
            nombre = element.text
           
            if nombre not in dict_actores.keys():
                dict_actores[nombre]  = element.get_attribute("href")

                driver.find_element("css selector",'section[data-testid="title-cast"]')


In [24]:
dict_actores


{'Jamie Harris': 'https://www.imdb.com/name/nm0364807/?ref_=tt_cl_t_1',
 'Camilla Enders': 'https://www.imdb.com/name/nm0256813/?ref_=tt_cl_t_2',
 'Denis Holmes': 'https://www.imdb.com/name/nm0391799/?ref_=tt_cl_t_3',
 'Layla Alexander': 'https://www.imdb.com/name/nm0018916/?ref_=tt_cl_t_4',
 'Olga Alexandrova': 'https://www.imdb.com/name/nm14869118/?ref_=tt_cl_t_5',
 'Michele Ammon': 'https://www.imdb.com/name/nm1994376/?ref_=tt_cl_t_6',
 'Tyler Andrews': 'https://www.imdb.com/name/nm14869119/?ref_=tt_cl_t_7',
 'Dann R. Black': 'https://www.imdb.com/name/nm14869117/?ref_=tt_cl_t_8',
 'Valerie Bodoy': 'https://www.imdb.com/name/nm14869122/?ref_=tt_cl_t_9',
 'Jean Pierre Brax': 'https://www.imdb.com/name/nm14869114/?ref_=tt_cl_t_10'}

In [20]:
##Busqueda info actores 
driver = webdriver.Chrome()

for link_actor in dict_actores["url"]:

    link = link_actor

    driver.get(link)

    driver.maximize_window()

    sleep(5)

    # Aceptar las cookies PAG IMDB
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

    sleep(5)

    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"}
  (Session info: chrome=122.0.6261.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66A4EAD22+56930]
	(No symbol) [0x00007FF66A45F622]
	(No symbol) [0x00007FF66A3142E5]
	(No symbol) [0x00007FF66A3598ED]
	(No symbol) [0x00007FF66A359A2C]
	(No symbol) [0x00007FF66A39A967]
	(No symbol) [0x00007FF66A37BCDF]
	(No symbol) [0x00007FF66A3981E2]
	(No symbol) [0x00007FF66A37BA43]
	(No symbol) [0x00007FF66A34D438]
	(No symbol) [0x00007FF66A34E4D1]
	GetHandleVerifier [0x00007FF66A866AAD+3709933]
	GetHandleVerifier [0x00007FF66A8BFFED+4075821]
	GetHandleVerifier [0x00007FF66A8B817F+4043455]
	GetHandleVerifier [0x00007FF66A589756+70

In [92]:
list_busqueda_rotten[:5]

[{'Chateaubriand - Cabeça de Paraíba': 2000},
 {'Sé quién eres': 2000},
 {'La chambre obscure': 2000},
 {'Shadow Magic': 2000},
 {'A Christmas Tree and a Wedding': 2000}]

In [105]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()
sleep(3)

puntuacion_tomatoes = {"Rotten Tomatoes":[]}
lista_titulos

for peli in list_busqueda_rotten[:4]:
    for titulo,año in peli.items():
         
        busqueda = f"TOMATOMETER title: {titulo} - year: {año}"
        # print(busqueda)
        
        try:
                       
            buscador = driver.find_element("css selector",'#APjFqb')
            buscador.clear()
            buscador.send_keys(busqueda, Keys.ENTER)
            sleep(random.randint(4,8))

            try:
                puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:first-child > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
                                
            except:
                puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes["Rotten Tomatoes"].append(puntuacion.group())
        
        except:
            puntuacion_tomatoes["Rotten Tomatoes"].append("No existe puntuacion")
        


In [106]:
print(puntuacion_tomatoes["Rotten Tomatoes"])

['No existe puntuacion', 'No existe puntuacion', 'No existe puntuacion', '72%']
